In [7]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import tkinter as tk
from tkinter import filedialog, Label
from PIL import Image, ImageTk

# File paths
csv_path = r"emotions.csv"  # CSV containing gender, age, country columns
images_folder = r"images"  # Folder containing subfolders 00, 01, etc.

# Load the data from CSV
data = pd.read_csv(csv_path)

# Preprocess images from nested folder structure
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (64, 64)).flatten()
    return img

# Collect image features and labels for nationality classification
X_images = []
y_nationality = []

# Iterate through the dataset and associate images with their labels
for i, row in data.iterrows():
    nationality = row['country']
    
    # Search for an image in the nested folder structure
    image_found = False
    for folder in os.listdir(images_folder):
        img_folder = os.path.join(images_folder, folder)
        if os.path.isdir(img_folder):
            for img_file in os.listdir(img_folder):
                img_path = os.path.join(img_folder, img_file)
                if os.path.isfile(img_path):
                    features = preprocess_image(img_path)
                    if features is not None:
                        X_images.append(features)
                        y_nationality.append(nationality)
                        image_found = True
                    break
        if image_found:
            break

# Ensure we have enough data to proceed
if len(X_images) == 0:
    raise ValueError("No images found. Please check your images_folder and file paths.")

# Convert lists to numpy arrays
X_images = np.array(X_images)
y_nationality = np.array(y_nationality)

# Ensure we have enough samples for train/test split
if len(X_images) < 2:
    raise ValueError("Not enough samples for training. Add more images.")

# Split data and train nationality classifier
X_train, X_test, y_train, y_test = train_test_split(X_images, y_nationality, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a RandomForest model for nationality classification
nationality_model = RandomForestClassifier()
nationality_model.fit(X_train, y_train)

# Dummy functions for additional attributes (replace with real models if needed)
def predict_age():
    return np.random.randint(20, 50)

def predict_color():
    return np.random.choice(['red', 'blue', 'green'])

def predict_emotion():
    return np.random.choice(['happy', 'sad', 'neutral'])

# Prediction logic based on nationality
def predict_attributes(nationality, img_path):
    if nationality == "Indian":
        age = predict_age()
        color = predict_color()
        emotion = predict_emotion()
        return f"Nationality: {nationality}\nAge: {age}\nDress Color: {color}\nEmotion: {emotion}"
    elif nationality == "United States":
        age = predict_age()
        emotion = predict_emotion()
        return f"Nationality: {nationality}\nAge: {age}\nEmotion: {emotion}"
    elif nationality == "African":
        color = predict_color()
        emotion = predict_emotion()
        return f"Nationality: {nationality}\nDress Color: {color}\nEmotion: {emotion}"
    else:
        emotion = predict_emotion()
        return f"Nationality: {nationality}\nEmotion: {emotion}"

# GUI Functions
def upload_image():
    filepath = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    if filepath:
        img = Image.open(filepath)
        img = img.resize((200, 200))
        img = ImageTk.PhotoImage(img)
        image_label.config(image=img)
        image_label.image = img
        
        # Predict nationality and additional attributes
        image_features = preprocess_image(filepath)
        if image_features is not None:
            image_features = scaler.transform([image_features])
            nationality = nationality_model.predict(image_features)[0]
            result_text = predict_attributes(nationality, filepath)
            output_label.config(text=result_text)
        else:
            output_label.config(text="Error: Could not process the image.")



In [9]:
# GUI Setup
root = tk.Tk()
root.title("Nationality Detection Model")
root.geometry("400x500")

upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack()

image_label = Label(root)
image_label.pack()

output_label = tk.Label(root, text="Results will be shown here", wraplength=300)
output_label.pack()

root.mainloop()
